### Use BERT to score a blank-filling option
### Model based on BERT_Cloze

## 1. Loading and Initializing

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
print("Using Tensorflow version: " + tf.__version__)
print(tf.config.list_physical_devices('GPU'))

BERT_DIR = "/home/aufish/Downloads/bert"

Using Tensorflow version: 2.1.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
bert_module = hub.KerasLayer(BERT_DIR, trainable=True)

In [3]:
# tokenizer
from bert import tokenization

def create_tokenizer(vocab_file, do_lower_case=False):
    return tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer(BERT_DIR + "/assets/vocab.txt")

def convert_sentence_to_features(sentence, tokenizer, max_seq_len=50):
    tokens = ['[CLS]']
    tokens.extend(tokenizer.tokenize(sentence))
    if len(tokens) > max_seq_len-1:
        tokens = tokens[:max_seq_len-1]
    tokens.append('[SEP]')+63
    
    segment_ids = [0] * len(tokens)
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_ids)

    #Zero Mask till seq_length
    zero_mask = [0] * (max_seq_len-len(tokens))
    input_ids.extend(zero_mask)
    input_mask.extend(zero_mask)
    segment_ids.extend(zero_mask)
    
    return input_ids, input_mask, segment_ids

def convert_sentences_to_features(sentences, tokenizer, max_seq_len=50):
    all_input_ids = []
    all_input_mask = []
    all_segment_ids = []
    
    for sentence in sentences:
        input_ids, input_mask, segment_ids = convert_sentence_to_features(sentence, tokenizer, max_seq_len)
        all_input_ids.append(input_ids)
        all_input_mask.append(input_mask)
        all_segment_ids.append(segment_ids)
    
    return all_input_ids, all_input_mask, all_segment_ids

import random, copy
import numpy as np
MASK_ID = tokenizer.convert_tokens_to_ids(['[MASK]'])[0]
def make_mask(input_ids, input_mask, mask_loc):
    ''' 
    Only make mask for one sentence
    input_ids: the ids of words in the sentences
    input_mask: initial mask (1 if there is a word; 0 for padding)
    mask_loc: which word should be masked, notice that the first in-sentence word has index 1
    returns
    input_id: the word specified by mask_loc is replaced by [MASK]
    input_mask: replace one bit of 1 with 0, meaning that the word will be masked
    original_word: the word that is replaced
    '''
    
    new_input_mask = copy.deepcopy(input_mask)
    new_input_ids  = copy.deepcopy(input_ids)
    
    original_word = input_ids[mask_loc]
    
    new_input_ids[mask_loc] = MASK_ID
    
    assert new_input_mask[mask_loc] == 1
    new_input_mask[mask_loc] = 0
                
    return new_input_ids, new_input_mask, original_word

### 1.2 Blank filler model (from BERT_Cloze)

In [4]:
class WordPredictor(tf.keras.Model):
    # The output means, how possible the given word may fit into the blank
    def __init__(self, class_num, bert=bert_module, dropout=0.1):
        super(WordPredictor, self).__init__()
        self.bert = bert
        self.drop = tf.keras.layers.Dropout(rate=dropout, trainable=True)
        
        self.dense = tf.keras.layers.Dense(
            class_num,
            activation=None,
            kernel_initializer='glorot_uniform',
            name='word_prediction',
            trainable=True)
        
    def call(self, inputs, mask_loc):
        # When passed in, all tensors are stacked in one, split it into a list
        inputs = tf.unstack(tf.cast(inputs, tf.dtypes.int32), axis=1)

        pooled, sequential = self.bert(inputs)
        
        # select one from each batch
        s = tf.gather_nd(sequential, [(i, mask_loc[i]) for i in range(sequential.shape[0])])
        # s now has shape (batch_size * 768)
        
        x = self.drop(s)
        return self.dense(x)

## 2. Scorer

In [ ]:
def score_a_sentence(model, sentence):
    # Given a sentence, use WordPredictor to estimate how good each word is
    